In [7]:
# import torchinfo
import torch.nn as nn
import torch

In [23]:
class Simple_Auto_NN(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.network = nn.Sequential(
        nn.Linear(num_features, 3),
        nn.ReLU(),
        nn.Linear(3, 1),
        nn.Sigmoid())
        
    def forward(self, features):
        out = self.network(features)
        return out


In [24]:
features = torch.rand(10, 5)
model = Simple_Auto_NN(features.shape[1])
model(features)

tensor([[0.6232],
        [0.6177],
        [0.6328],
        [0.6179],
        [0.6357],
        [0.6254],
        [0.6266],
        [0.6269],
        [0.6287],
        [0.6229]], grad_fn=<SigmoidBackward0>)

In [27]:
model.network.weights

AttributeError: 'Sequential' object has no attribute 'weights'

In [16]:
!pip install torchinfo

In [28]:
from torchinfo import summary
summary(model, input_size = features.shape)

Layer (type:depth-idx)                   Output Shape              Param #
Simple_Auto_NN                           [10, 1]                   --
├─Sequential: 1-1                        [10, 1]                   --
│    └─Linear: 2-1                       [10, 3]                   18
│    └─ReLU: 2-2                         [10, 3]                   --
│    └─Linear: 2-3                       [10, 1]                   4
│    └─Sigmoid: 2-4                      [10, 1]                   --
Total params: 22
Trainable params: 22
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

Automating previous manual code

In [30]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

In [31]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.drop(columns = ['id', 'Unnamed: 32'], inplace = True)
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [32]:
X_train, X_test, Y_train, Y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size = 0.2)

In [33]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [34]:
encoder = LabelEncoder()
Y_train = encoder.fit_transform(Y_train)
Y_test = encoder.fit_transform(Y_test)

In [35]:
X_train = torch.tensor(X_train)
X_test = torch.tensor(X_test)
Y_train = torch.tensor(Y_train)
Y_test = torch.tensor(Y_test)


In [ ]:
class AutoNN(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.linear = nn.Linear(num_features, 1, dtype = torch.float64)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, features):
        out = self.linear(features)
        out = self.sigmoid(out)
        return out

   

In [46]:
lr = 0.1
epochs = 25

In [56]:
loss_function = nn.BCELoss()

In [53]:
type(loss_function)

torch.nn.modules.loss.BCELoss

In [61]:
# create model
model = AutoNN(X_train.shape[1])
X_train = X_train.double()
Y_train = Y_train.double()
optimizer = torch.optim.SGD(model.parameters(), lr = lr)
# define loop
for epoch in range(epochs):

  # forward pass
  y_pred = model(X_train)

  # loss calculate
  loss = loss_function(y_pred, Y_train.view(-1, 1))

  # backward pass
  loss.backward()

  # parameters update
  optimizer.step()
  # zero gradients
  optimizer.zero_grad()
  # print loss in each epoch
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 0.7553379004621388
Epoch: 2, Loss: 0.5562634558799351
Epoch: 3, Loss: 0.4495758466030083
Epoch: 4, Loss: 0.3859327663376969
Epoch: 5, Loss: 0.34383903984415565
Epoch: 6, Loss: 0.31373939588056676
Epoch: 7, Loss: 0.29094473567344203
Epoch: 8, Loss: 0.2729269912699598
Epoch: 9, Loss: 0.2582134795662313
Epoch: 10, Loss: 0.24589053943654623
Epoch: 11, Loss: 0.23536126093891976
Epoch: 12, Loss: 0.22621859215406306
Epoch: 13, Loss: 0.2181747925241053
Epoch: 14, Loss: 0.21102016929691633
Epoch: 15, Loss: 0.20459786866310659
Epoch: 16, Loss: 0.19878788614668724
Epoch: 17, Loss: 0.19349658864786223
Epoch: 18, Loss: 0.1886496496790999
Epoch: 19, Loss: 0.18418716472395488
Epoch: 20, Loss: 0.18006019772884574
Epoch: 21, Loss: 0.17622829017954636
Epoch: 22, Loss: 0.172657631849768
Epoch: 23, Loss: 0.16931969536788397
Epoch: 24, Loss: 0.1661902017267163
Epoch: 25, Loss: 0.16324832577514495


In [48]:
model.linear.weight

Parameter containing:
tensor([[-0.0298, -0.0056,  0.0198, -0.1513, -0.0752,  0.1071,  0.0200,  0.0954,
         -0.0798,  0.0127,  0.0681,  0.0938,  0.0553, -0.0254, -0.0704, -0.0913,
          0.1204, -0.0691,  0.0536, -0.0706,  0.1037, -0.1107, -0.0371,  0.0892,
          0.0013, -0.1281, -0.0577, -0.0900,  0.1152,  0.1571]],
       dtype=torch.float64, requires_grad=True)

In [62]:
with torch.no_grad():
    y_pred = model.forward(X_test)
    y_pred = (y_pred > 0.9).float()
    accuracy = (y_pred == Y_test).float().mean()
    print(f"Accuracy:{accuracy.item()}")

Accuracy:0.5763311982154846
